# Calcul du tenseur d'Eshelby dans le cas des ellipses

## 0 Quelques fonctions utiles

In [1]:
import numpy as np
from numpy import pi
from numpy.random import random_sample
from numpy.linalg import inv
from numpy import dot
from scipy.spatial.transform import Rotation as R 
from classes import *

def Comp3333_to_66 (G) : 
    "Passe d'un tenseur de comportement  G 3x3x3x3 a une matrice de comportement F 6x6"
    F=np.zeros((6,6))
    for i in range(3):
        for j in range(3):
            F[i,j] = G[i,i,j,j]
            
        F[i,5]=(G[i,i,0,1]+G[i,i,1,0])/2.
        F[i,3]=(G[i,i,1,2]+G[i,i,2,1])/2. 
        F[i,4]=(G[i,i,2,0]+G[i,i,0,2])/2. 
        F[3,i]=(G[1,2,i,i]+G[2,1,i,i])/2. 
        F[4,i]=(G[0,2,i,i]+G[2,0,i,i])/2.
        F[5,i]=(G[0,1,i,i]+G[1,0,i,i])/2.

    F[4,4]=(G[0,2,0,2]+G[2,0,0,2]+G[0,2,2,0]+G[2,0,2,0])/4. 
    F[3,3]=(G[1,2,1,2]+G[2,1,1,2]+G[1,2,2,1]+G[2,1,2,1])/4.  
    F[5,5]=(G[0,1,0,1]+G[1,0,0,1]+G[0,1,1,0]+G[1,0,1,0])/4.  
    F[4,3]=(G[0,2,1,2]+G[2,0,1,2]+G[0,2,2,1]+G[2,0,2,1])/4.  
    F[4,5]=(G[0,2,1,0]+G[2,0,1,0]+G[0,2,0,1]+G[2,0,0,1])/4.  
    F[3,4]=(G[1,2,0,2]+G[2,1,0,2]+G[1,2,2,0]+G[2,1,2,0])/4.  
    F[5,4]=(G[0,1,0,2]+G[1,0,0,2]+G[0,1,2,0]+G[1,0,2,0])/4.  
    F[3,5]=(G[1,2,1,0]+G[2,1,1,0]+G[1,2,0,1]+G[2,1,0,1])/4.   
    F[5,3]=(G[0,1,1,2]+G[1,0,1,2]+G[0,1,2,1]+G[1,0,2,1])/4. 
    
    return F

def Comp66_to_3333(F) : 
    'Passe d une matrice F 6x6 à un tenseur G 3x3x3x3'
    G = np.zeros((3,3,3,3))
    for i in range(3) :
        for j in range(3) :
            G[i,i,j,j]=F[i,j]
       
        G[i,i,0,1]=F[i,5]
        G[i,i,1,2]=F[i,3]
        G[i,i,2,0]=F[i,4]
        G[0,2,i,i]=F[4,i]
        G[1,2,i,i]=F[3,i]
        G[0,1,i,i]=F[5,i]
        G[i,i,1,0]=F[i,5]
        G[i,i,2,1]=F[i,3]
        G[i,i,0,2]=F[i,4]
        G[2,0,i,i]=F[4,i]
        G[2,1,i,i]=F[3,i]
        G[1,0,i,i]=F[5,i]
        
    G[0,1,0,1]=F[5,5]
    G[0,1,0,2]=F[5,4]
    G[0,1,1,0]=F[5,5]
    G[0,1,1,2]=F[5,3] 
    G[0,1,2,0]=F[5,4]
    G[0,1,2,1]=F[5,3]

    G[0,2,0,1]=F[4,5]
    G[0,2,0,2]=F[4,4]
    G[0,2,1,0]=F[4,5]
    G[0,2,1,2]=F[4,3] 
    G[0,2,2,0]=F[4,4]
    G[0,2,2,1]=F[4,3]

    G[1,0,0,1]=F[5,5]
    G[1,0,0,2]=F[5,4]
    G[1,0,1,0]=F[5,5]
    G[1,0,1,2]=F[5,3] 
    G[1,0,2,0]=F[5,4]
    G[1,0,2,1]=F[5,3]

    G[1,2,0,2]=F[3,4]
    G[1,2,1,0]=F[3,5]
    G[1,2,1,2]=F[3,3] 
    G[1,2,2,0]=F[3,4]
    G[1,2,2,1]=F[3,3]

    G[2,0,0,1]=F[4,5]
    G[2,0,0,2]=F[4,4]
    G[2,0,1,0]=F[4,5]
    G[2,0,1,2]=F[4,3] 
    G[2,0,2,0]=F[4,4]
    G[2,0,2,1]=F[4,3]

    G[2,1,0,1]=F[3,5]
    G[2,1,0,2]=F[3,4]
    G[2,1,1,0]=F[3,5]
    G[2,1,1,2]=F[3,3] 
    G[2,1,2,0]=F[3,4]
    G[2,1,2,1]=F[3,3]
 
    return G 

def Rotation_angles():
    theta = 2*pi*np.random.random()
    psi = 2*pi*np.random.random()
    phi = np.arccos(np.random.random())
    return (phi,theta,psi)


def Matrice_rotation(psi,phi,theta) : 
    'Crée une matrice de rotation 3x3 à partir des trois angles d euler'
    Q = np.zeros((3,3))
    
    Q[0,0]=cos(psi)*cos(theta)-cos(phi)*sin(theta)*sin(psi)
    Q[0,1]=sin(theta)*cos(psi)+cos(phi)*sin(psi)*cos(theta)
    Q[0,2]=sin(phi)*sin(psi)
    Q[1,0]=-sin(psi)*cos(theta)-sin(theta)*cos(phi)*cos(psi)
    Q[1,1]=cos(psi)*cos(phi)*cos(theta)-sin(theta)*sin(psi)
    Q[1,2]=cos(psi)*sin(phi)
    Q[2,0]=sin(phi)*sin(theta)
    Q[2,1]=-sin(phi)*cos(theta)
    Q[2,2]=cos(phi)
    
    for i in range(3) : 
        for j in range(3):
            if (abs(Q[i,j]) < 10**-6 ) :
                Q[i,j] = 0
            
    return Q

def Rotation_tenseur(S,phi,theta,psi) : 
    ' Renvoie la rotation du tenseur S par les 3 angles d Euler'
    B = np.zeros((3,3,3,3))
    R = Matrice_rotation(psi,phi,theta)
    for  i in range(3) : 
        for  j in range(i+1):
            for  k in range(3):
                for  l in range(k+1):
                    for  m in range(3):
                        for  n in range(3):
                            for  ll in range(3):
                                for  kk in range(3):
                                    B[i,j,k,l] += R[i,m]*R[j,n]*R[k,ll]*R[l,kk]*S[m,n,ll,kk]
                                    B[i,j,l,k] = B[i,j,k,l]
                                    B[j,i,k,l] = B[i,j,k,l]
                                    B[j,i,l,k] = B[i,j,k,l]
    return B


def Matrice_Souplesse_Isotrope(E,nu) :
    'Renvoie la matrice de souplesse d un matériau isotrope'
    S = np.zeros((6,6))
    S[0,0]=1./E
    S[1,1]=1./E
    S[2,2]=1./E

    S[3,3]=2.*(1+nu)/E
    S[4,4]=2.*(1+nu)/E
    S[5,5]=2.*(1+nu)/E

    S[0,1]=-nu/E
    S[0,2]=-nu/E
    S[1,2]=-nu/E
    S[1,0]=-nu/E
    S[2,1]=-nu/E
    S[2,0]=-nu/E
    
    return S
    

def Young_isotrope(S) : 
    return 3/(S[0,0]+S[1,1]+S[2,2])

def nu_isotrope(S) : 
    Ey = 1/S[1,1]
    Ez = 1/S[2,2]
    return - 1/3 * (Ey*S[0,1] + Ez*S[0,2] + Ez*S[1,2])
    
def Young_anisotrope(C) : 
    return C[0,0],C[1,1],C[2,2]


def Compute_with_permutation(a,I,II,nu) : 
    S = np.zeros((3,3,3,3))
    
    for i in range(3) :
        S[i,i,i,i] = 3*a[i]**2*II[i][i] / (8*pi*(1-nu)) + I[i] * (1-2*nu)/(8*pi*(1-nu))
        j = (i+1)%3
        S[i,i,j,j] = a[j]**2*II[i][j]/(8*pi*(1-nu)) -  I[i] * (1-2*nu)/(8*pi*(1-nu))
        S[i,j,i,j] = (a[i]**2+a[j]**2)*II[i][j]/(16*pi*(1-nu)) + (1-2*nu)/(16*pi*(1-nu))*(I[i]+I[j])
        k = (i+2)%3
        S[i,i,k,k] = a[k]**2*II[i][k]/(8*pi*(1-nu)) -  I[i] * (1-2*nu)/(8*pi*(1-nu))
        S[i,k,i,k] = (a[i]**2+a[k]**2)*II[i][k]/(16*pi*(1-nu)) + (1-2*nu)/(16*pi*(1-nu))*(I[i]+I[k])
        
    return S

def Cyclic_permutation(S) : 
    for i in range(3) : 
        for j in range(3) :
            for k in range(3) : 
                for l in range(3) : 
                    val = non_zeros(S,i,j,k,l)
                    S[i,j,k,l] = val
                    S[i,j,l,k] = val
                    S[j,i,k,l] = val
                    S[j,i,l,k] = val
    return S
                    

def non_zeros(S,i,j,k,l) : 
    if S[i,j,k,l] != 0 : 
        return S[i,j,k,l]
    if S[i,j,l,k] != 0 : 
        return S[i,j,l,k]
    if S[j,i,k,l] != 0 : 
        return S[j,i,k,l]
    if S[j,i,l,k] != 0 : 
        return S[j,i,l,k]
    return 0

def clear_matrix3 (C,k) : 
    n = C.shape[0]
    for i in range(n) : 
        for j in range(n) :
            if C[i,j,k]<10**-8 : 
                C[i,j,k] = 0
                
def clear_matrix2 (C) : 
    n = C.shape[0]
    for i in range(n) : 
        for j in range(n) :
            if C[i,j]<10**-8 : 
                C[i,j] = 0
    

# Calcul du tenseur d'Eshelby dans le cas général

In [2]:
def Eshelby_tensor(Axis,Em,nu) : 
    
    Sm = Matrice_Souplesse_Isotrope(Em,nu)
    Cm = inv(Sm)
    Cm3 = Comp66_to_3333(Cm)
    a0,a1,a2 = Axis
    IJV = np.array([[0,0],[1,1],[2,2],[1,2],[0,2],[0,1]])
    Nit = 40
    Ntop = Nit
    Mtop = Nit
    dphi = pi/(Ntop-1)
    dtheta = pi/(Ntop-1)
    A = np.zeros((6,6))
    B = np.zeros((6,6,Mtop))
    G = np.zeros((6,6,Ntop))
    E = np.zeros((6,6))
    
    # Integration de la fonction de green sur la demi ellipsoïde
    for m in range(Mtop) : 
        phi = m*dphi
        for n in range(Ntop) : 
            theta = n*dtheta
            X = np.array([sin(theta)*cos(phi)/a0 , sin(theta)*sin(phi)/a1 , cos(theta)/a2])
            CXX = np.zeros((3,3))
            for i in range(3) :
                for j in range(3) :
                    for k in range(3) : 
                        for l in range(3) :
                            CXX[i,k] += Cm3[i,j,k,l]*X[j]*X[l]
            CXX = inv(CXX)
            for i in range(6) :
                for j in range(6) :                     
                    I1 = IJV[i,0]
                    J1 = IJV[j,0]
                    I2 = IJV[i,1]
                    J2 = IJV[j,1]
                    G[i,j,n] = 0.5 * sin(theta) * (CXX[I1,J1]*X[I2]*X[J2] + CXX[I2,J1]*X[I1]*X[J2] + CXX[I1,J2]*X[I2]*X[J1] + CXX[I2,J2]*X[I1]*X[J1])
        
        
        B[:,:,m] = 0.5 * dtheta * (G[:,:,0]+G[:,:,Ntop-1])
        for i in range(1,Ntop-1) : 
            B[:,:,m] +=  dtheta * G[:,:,i]

    A = 0.5*(B[:,:,0]+B[:,:,Ntop-1])* dphi/(4*pi)
    for i in range(1,Ntop-1) : 
         A += B[:,:,i]* dphi/(4*pi)  
    
    for i in range(6) : 
        for j in range(6) : 
            E[i,j]=A[i,0]*Cm[0,j]+A[i,1]*Cm[1,j]+A[i,2]*Cm[2,j] + 4* (A[i,3]*Cm[3,j]+A[i,4]*Cm[4,j]+A[i,5]*Cm[5,j]) 
    
    return E

In [3]:
E = Eshelby_tensor((1,1,1),1,0)
clear_matrix2(E)
print(E)

[[0.46666702 0.         0.         0.         0.         0.        ]
 [0.         0.46666702 0.         0.         0.         0.        ]
 [0.         0.         0.46612587 0.         0.         0.        ]
 [0.         0.         0.         0.53279201 0.         0.        ]
 [0.         0.         0.         0.         0.53279201 0.        ]
 [0.         0.         0.         0.         0.         0.53333369]]


## MODELE AUTOCOHERENT UNE FORME D'INCLUSION

In [4]:
from scipy.spatial.transform import Rotation as Rot
        
def compute_h_behavior(A,f):
    
    n_renforts = 1000     # paramètre non physique qui permet de forcer lisotropie
    n_pas = 8            # pas du modèle autocohérent
    precision = 10**-2  # précision désirée dans l'algorithme du point fixe
    

    # Création des matrices de comportement
    Em = 1
    num = 0
    Ef = 100
    nuf = 0
    
    Sm = Matrice_Souplesse_Isotrope(Em,num)
    Cm = inv(Sm)
    Sf = Matrice_Souplesse_Isotrope(Ef,nuf)
    Cf = inv(Sf)    
    Id = np.identity(6)
    
    #Initialisation du point fixe
    Cp = Cm
    Sp = Sm
    
    # Boucle d'incrémentation de la fraction volumique
    for i in range(n_pas+1) : 
        f_pas = f*i/n_pas
        f_1_renfort = f_pas/n_renforts
        
        # Algorithme du point fixe : Ch=f(Ch) pour f fixé
        
        convergence = 2
        
        Eh = Young_isotrope(Sp)
        nuh = nu_isotrope(Sp)
        
        print('f_pas : ',f_pas)
        print('        convergence',convergence)
        while convergence>precision :   
            W = np.zeros((6,6)) # Matrice des contributions de l'inclusion dans Ch
            Esh = Eshelby_tensor(A,Eh,nuh)
            #print('Esh')
            #print(Esh)
            Aesh = inv(Id + np.matmul(Esh,np.matmul(Sp,Cf-Cp)))
            #print('Aesh')
            #print(Aesh)
            V6 = np.dot(Cf-Cm,Aesh)
            #print('V6')
            #print(V6)
            V3 = Comp66_to_3333(V6)
            # Ajout des contribution de chaque renfort en fonction de son orientation
            for i in range(n_renforts) : 
                theta,phi,psi = Rot.random().as_euler('zxy', degrees=False)
                #phi,theta,psi = Rotation_angles()
                #phi,theta,psi = 0,0,0
                V3R = Rotation_tenseur(V3,phi,theta,psi)
                V = Comp3333_to_66(V3R)
                W += f_1_renfort * V
            #print('V3')
            #print(V3)
            #print('V3R')
            #print(V3R)
            #print('Rot')
            #print( Matrice_rotation(psi,phi,theta))
            #print('W')
            #print(W)
            Ch = Cm + W
            # Actualisation du matériau homogénéisé
            Cp = Ch
            Sp = inv(Cp)
            
            # Test de sortie
            E = Young_isotrope(Sp)
            nu = nu_isotrope(Sp)
            convergence = abs(E-Eh)
            print('        convergence',convergence)
            Eh = E
            nuh = nu           
            
            # Forçage de la matrice en matrice isotrope
            Sp = Matrice_Souplesse_Isotrope(Eh,nuh)
            Cp = inv(Sp)
        print('Eh = ',Eh)
        print('nuh = ',nuh)

    return Cp

In [ ]:
A = (3,2,1)
f = 0.4
print(compute_h_behavior(A,f))

f_pas :  0.0
        convergence 2
        convergence 0.0
Eh =  1.0
nuh =  -0.0
f_pas :  0.05
        convergence 2


## MODELE AUTOCOHERENT PLUSIEURS FORMES D'INCLUSION

In [6]:
def compute_h_behavior_multi(Argument):
    
    n_renforts = 100     # paramètre non physique qui permet de forcer lisotropie
    n_pas = 2            # pas du modèle autocohérent
    precision = 10**-2  # précision désirée dans l'algorithme du point fixe
    

    # Création des matrices de comportement
    Em = 1
    num = 0
    Ef = 100
    nuf = 0
    
    Sm = Matrice_Souplesse_Isotrope(Em,num)
    Cm = inv(Sm)
    Sf = Matrice_Souplesse_Isotrope(Ef,nuf)
    Cf = inv(Sf)    
    Id = np.identity(6)
    
    #Initialisation du point fixe
    Cp = Cm
    Sp = Sm
    
    n_inclusions = len(Argument)
    A = np.zeros((n_inclusions,3))
    F = np.zeros(n_inclusions)
    for i in range(n_inclusions) : 
        A[i,:],F[i] = Argument[i]
    
    # Boucle d'incrémentation du pas
    for i in range(n_pas+1) : 
        F_pas = F*i/n_pas
        F_1_renfort = F_pas/n_renforts
        
        # Algorithme du point fixe : Ch=f(Ch) pour f fixé
        
        convergence = 2
        
        Eh = Young_isotrope(Sp)
        nuh = nu_isotrope(Sp)
        
        print('f_pas : ',F_pas)
        print('convergence',convergence)
        while convergence>precision :   
            W = np.zeros((6,6))           # Matrice des contributions de l'inclusion dans Ch
            # Boucle sur les différentes formes d'inclusions
            for j in range(n_inclusions) : 
                Esh = Eshelby_tensor(A[j],Eh,nuh)
                #print('Esh')
                #print(Esh)
                Aesh = inv(Id + np.matmul(Esh,np.matmul(Sp,Cf-Cp)))
                #print('Aesh')
                #print(Aesh)
                V6 = np.dot(Cf-Cm,Aesh)
                #print('V6')
                #print(V6)
                V3 = Comp66_to_3333(V6)
                # Ajout des contribution de chaque renfort en fonction de son orientation
                for i in range(n_renforts) : 
                    theta,phi,psi = Rot.random().as_euler('zxy', degrees=False)
                    #phi,theta,psi = Rotation_angles()
                    #phi,theta,psi = 0,0,0
                    V3R = Rotation_tenseur(V3,phi,theta,psi)
                    V = Comp3333_to_66(V3R)
                    W += F_1_renfort[j] * V
            #print('V3')
            #print(V3)
            #print('V3R')
            #print(V3R)
            #print('Rot')
            #print( Matrice_rotation(psi,phi,theta))
            #print('W')
            #print(W)
            Ch = Cm + W
            # Actualisation du matériau homogénéisé
            Cp = Ch
            Sp = inv(Cp)
            
            # Test de sortie
            E = Young_isotrope(Sp)
            nu = nu_isotrope(Sp)
            convergence = abs(E-Eh)
            #print('E', E,' nu',nu)
            #print('Eh', Eh,' nuh',nuh)
            print('convergence',convergence)
            Eh = E
            nuh = nu           
            
            # Forçage de la matrice en matrice isotrope
            Sp = Matrice_Souplesse_Isotrope(Eh,nuh)
            Cp = inv(Sp)
            
    return Cp


### TEST

In [7]:
A = (3,2,1)
f = 0.2/3
Argument = [(A,f),(A,f),(A,f)]
Ch = compute_h_behavior_multi(Argument)
print('Ch')
print(Ch)

f_pas :  [0. 0. 0.]
convergence 2
convergence 0.0
f_pas :  [0.03333333 0.03333333 0.03333333]
convergence 2
convergence 0.25320320496455695
convergence 0.05753770967945737
convergence 0.011696719720124982
convergence 0.003625280870453107
f_pas :  [0.06666667 0.06666667 0.06666667]
convergence 2
convergence 0.6570982804851093
convergence 0.12786100133259604
convergence 0.05771124285277485
convergence 0.022006576777925613
convergence 0.011842687250456452
convergence 0.004589859453542955
Ch
[[1.90686237 0.1632658  0.1632658  0.         0.         0.        ]
 [0.1632658  1.90686237 0.1632658  0.         0.         0.        ]
 [0.1632658  0.1632658  1.90686237 0.         0.         0.        ]
 [0.         0.         0.         0.87179829 0.         0.        ]
 [0.         0.         0.         0.         0.87179829 0.        ]
 [0.         0.         0.         0.         0.         0.87179829]]
